In [1]:
# libraries
import pandas as pd

# classification dictionary
import os
import sys
sys.path.append(os.path.abspath("modules"))
import modules.dictionary as dict
from modules.classify import classify_keywords

# set your path here
os.chdir("input_data")
print(f'current directory: {os.getcwd()}')

current directory: /Users/tomokitakata/Desktop/research/dev/muldistribution/input_data


ベースの分類は東大病院の診療科構成を参考。  
- 腎臓・人工透析内科：これはこれで一つあるっぽい
- ペインクリニック：内科、麻酔科、整形外科に分類される場合がある。
- 平成 20 年 4 月 1 日以降、広告することが認められない診療科名「神経科」、「呼吸器科」、「消化器科」、「胃腸科」、「循環器科」、「皮膚泌尿器科」、「性病科」、「こう門科」、「気管食道科」

In [2]:
# override_map_2010 = {
#     "気管食道科": "呼吸器内科",
#     "リ": "リウマチ科",  # 市立札幌病院
#     "神経科": '脳神経外科',
#     "循環器科": '循環器内科',
#     '呼吸器科': '呼吸器内科',
# }

# partial matching
override_map_2014 = {

    # ----内科・外科----
    "訪問診療科": ["一般内科"],
    "訪問診療": ["一般内科"],
    "漢方": ["一般内科"], 
    "総合診療": ['一般内科', '一般外科'],
    "総合診療科": ['一般内科', '一般外科'],
    "基本診療": ['一般内科', '一般外科'],
    "基本診療科": ['一般内科', '一般外科'],

     # ----消化器内科・外科----
    "消科": ["消化器内科", "消化器外科"],
    "消化器": ["消化器内科", "消化器外科"],
    "消化器科": ["消化器内科", "消化器外科"],
    "消火器科": ['消化器内科', '消化器外科'],
    "胃腸科": ["消化器内科", "消化器外科"],
    "胃腸下科": ["消化器内科", "消化器外科"],
    "胃腸器科": ["消化器内科", "消化器外科"],
    "消・胃腸科": ["消化器内科", "消化器外科"],
    "消胃": ["消化器内科", "消化器外科"],
    "食道科": ["消化器内科", "消化器外科"],
    "消化器・胃腸科": ["消化器内科", '消化器外科'],
    "消化器胃腸科": ["消化器内科", '消化器外科'],

     # ----呼吸器内科・外科----
    "気管科": ["呼吸器内科", '呼吸器外科'],
    "呼吸器科": ["呼吸器内科", '呼吸器外科'],
    "気管支科": ['呼吸器内科'],
    "気管食科": ['呼吸器内科'],
    "気管食道科": ["呼吸器内科", '呼吸器外科', "消化器内科", "消化器外科"],
    "気道食道科": ["呼吸器内科", '呼吸器外科', "消化器内科", "消化器外科"],

     # ----循環器内科・外科----
    "循環器科": ["循環器内科", "循環器外科"],
    "循環器": ["循環器内科", "循環器外科"],
    "循環科": ["循環器内科", "循環器外科"],

    "循呼": ["呼吸器内科", '呼吸器外科', "循環器内科", "循環器外科"],

    # ----精神・心療----
    "精心内": ["精神神経科", "心療内科"],#
    "精心内": ["精神神経科"],#
    "子どものこころ診療科": ['心療内科'],
    "心身医療科": ['心療内科'],

    "女性不妊症": ['産婦人科'],
    "疼": ['麻酔科', '整形外科', 'リハビリテーション科'],
    "成分献血": ['病理診断科'],#
    
    # ----肝臓・胆嚢・膵臓----
    "肝臓科": ['肝・胆・膵内科', '肝・胆・膵外科'],
    "胆のう": ['肝・胆・膵内科', '肝・胆・膵外科'],
    "肝臓・胆のう・膵臓科": ['肝・胆・膵内科', '肝・胆・膵外科'],

    # ----癌----
    "が内": ['臨床腫瘍科'], # おそらく「がん内科」

    # ----歯科----
    "矯科": ["歯科・口腔顎顔面外科"], #おそらく「矯正歯科」
    "噛み合わせ不良による頭痛": ['歯科・口腔顎顔面外科'],

    # ----腎臓・内分泌----
    "血液浄化": ['腎臓・内分泌内科'],
    "高内": ["腎臓・内分泌内科"], # おそらく「高血圧・内分泌内科」
    "小腎": ['小児科', "腎臓・内分泌内科"], # おそらく 小児腎臓内科 or 小児腎臓外科
    "内高精": ["腎臓・内分泌内科", "泌尿器科"], # おそらく 内分泌・高血圧・生殖内科 or 生殖医療科 (精 = 精巣機能：男性不妊？)   
}

In [3]:
# 分類対象となるキーワードの例
uniq_dpt = pd.read_csv('2014.csv').iloc[:,1]
uniq_dpt = uniq_dpt.to_list()
uniq_dpt

['内科',
 '神経内科',
 '小児科',
 '外科',
 '整形外科',
 '脳神経外科',
 '呼吸器外科',
 '心臓血管外科',
 '小児外科',
 '泌尿器科',
 '産婦人科',
 '眼科',
 'リハビリテーション科',
 '放射線科',
 '麻酔科',
 '呼吸器内科',
 '循環器内科',
 '消化器内科',
 '消化器外科',
 '乳腺外科',
 '甲状腺外科',
 '病理診断科',
 '緩和ケア科',
 '精神科',
 '形成外科',
 '皮膚科',
 '耳鼻いんこう科',
 '歯科',
 '矯正歯科',
 '歯科口腔外科',
 'リウマチ科',
 '消化器科',
 '心療内科',
 '腫瘍外科',
 '血液内科',
 '腫瘍内科',
 '呼吸器科',
 '循環器科',
 'こう門科',
 '肛門外科',
 '腎臓内科',
 '糖尿病内科',
 '代謝内科',
 '緩和ケア内科',
 '内視鏡内科',
 '老年内科',
 '呼吸器リハビリテーション科',
 '児童精神科',
 '老年精神科',
 '大腸外科',
 '小児皮膚科',
 '内分泌内科',
 '神経科',
 '胃腸科',
 'アレルギー科',
 '産科',
 '婦人科',
 '脳神経内科',
 'こう門外科',
 '小児歯科',
 '人工透析内科',
 '小児神経科',
 '心臓血管内科',
 '糖尿病内分泌内科',
 '内視鏡外科',
 '頭頚部外科',
 '血管外科',
 '糖尿病・内分泌内科',
 '化学療法内科',
 '糖尿病・代謝内科',
 '血液・腫瘍内科',
 '胃腸内科（内視鏡）',
 'ペインクリニック内科',
 '内視鏡外科（大腸・胃）',
 '内分泌外科',
 '救急科',
 '放射線診断科',
 '胃腸内科',
 '循環器外科',
 '乳腺・内分泌内科',
 '呼吸気内科',
 '神経精神科',
 '人工透析外科',
 '気管食道科',
 '小児科（感染症・アレルギー疾患）',
 'ペインクリニック外科',
 '心臓内科',
 '腎臓内科（人工透析）',
 '老年脳神経外科',
 '放射線治療科',
 '胃・大腸・肛門外科',
 '糖尿病代謝内科',
 '腎臓移植外科',
 '頭頸部外科',
 '気管食道外科',
 '腫瘍整形外科',
 '胸部外科',
 

In [4]:
classified_keywords = classify_keywords(
    keywords=uniq_dpt,
    classification_dict=dict.classification_dict,
    multiple=True,            # 複数マッチを許可
    override_map=override_map_2014,
    override_match_type="partial"   # override判定は部分一致
)

# 分類結果をDataFrameに整形して表示
max_length = max(len(values) for values in classified_keywords.values())
data = {
    cat: values + [""] * (max_length - len(values))
    for cat, values in classified_keywords.items()
}
result_df = pd.DataFrame(data)
display(result_df)

,内科,一般内科,呼吸器内科,消化器内科,循環器内科,腎臓・内分泌内科,糖尿病・代謝内科,血液・腫瘍内科,アレルギー科,リウマチ科,...,小児外科,産婦人科,精神神経科,放射線科,救急・集中治療科,臨床腫瘍科,病理診断科,緩和ケア科,鍼灸院,分類不可
0,内科,漢方内科,呼吸器内科,消化器内科,循環器内科,腎臓内科,糖尿病内科,血液内科,アレルギー科,リウマチ科,...,小児外科,産婦人科,精神科,放射線科,救急科,腫瘍外科,病理診断科,緩和ケア科,指圧,透析科
1,神経内科,総合内科,呼吸器科,消化器内科,循環器内科,人工透析内科,代謝内科,腫瘍内科,小児科（感染症・アレルギー疾患）,リウマチ内科,...,小児心臓血管外科,産科,児童精神科,放射線診断科,小児救急科,腫瘍内科,理学診療科,緩和ケア内科,はり,不明
2,呼吸器内科,内科（消化器・循環器・呼吸器・内分泌・代謝・糖尿病・胃腸・大腸内視鏡・老人・人工透析・漢方）,呼吸気内科,消化器外科,循環器科,腎臓内科（人工透析）,糖尿病内分泌内科,心臓血管内科,内科（呼吸器・循環器・糖尿病・アレルギー疾患）,アレルギー疾患・リウマチ科,...,小児脳神経外科,婦人科,老年精神科,放射線治療科,,血液・腫瘍内科,病理診療科,疼痛緩和内科,,腎臓科
3,循環器内科,内科（血液・漢方）,気管食道科,消化器科,循環器科,内科（消化器・循環器・呼吸器・内分泌・代謝・糖尿病・胃腸・大腸内視鏡・老人・人工透析・漢方）,糖尿病・内分泌内科,血液・腫瘍内科,漢方アレルギー疾患皮膚科,アレルギー膠原病科,...,小児形成外科,婦人科（生殖医療）,神経精神科,腫瘍放射線科,,腫瘍整形外科,理学療法科,緩和医療科,,診療内
4,消化器内科,漢方アレルギー疾患皮膚科,内科（消化器・循環器・呼吸器・内分泌・代謝・糖尿病・胃腸・大腸内視鏡・老人・人工透析・漢方）,消化器科,循環器外科,透析内科,糖尿病・代謝内科,血液腫瘍内科,アレルギー疾患内科,リュウマチ科,...,小児科外科,産婦人科（産科）,小児精神科,神経放射線科,,血液腫瘍内科,病理診断・臨床検査科,緩和ケア,,医科
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,内科（代謝）,,,,,,,,,,...,,,,,,,,,,
466,内科（性感染症）,,,,,,,,,,...,,,,,,,,,,
467,内科（老年）,,,,,,,,,,...,,,,,,,,,,
468,内科（脂質代謝）,,,,,,,,,,...,,,,,,,,,,


In [16]:
# result_df.to_csv('../2010_classified.csv', encoding='utf-8', index=True)

診療科の分類は、医療レセプトにおける厚生労働省が指定する区分を使用。
https://www.mhlw.go.jp/topics/2009/05/dl/tp0521-1a_0053.pdf